# Pair Programming Limpieza V

En este caso trabajaremos con el dataframe que limpiamos en el ejercicio de pair de Limpieza III donde limpiabamos las columnas de species, age, etc. y eliminabamos los outliers. Hoy volveremos a gestionar valores nulos, pero en este caso usaremos los métodos de imputación de sklearn.

Es el momento de ponernos a trabajar con los valores nulos de nuevo 💪🏽. A lo largo de este ejercicio de pair programming vamos a intentar eliminar los valores nulos de nuestras columnas. En la lección hemos aprendido varios métodos de skelarn intentemos aplicarlos todos. Manos a la obra!

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sidetable
import warnings
warnings.filterwarnings('ignore')

In [7]:
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

In [29]:
df = pd.read_csv("datos/3.ataques_limpio.csv", index_col = 0)
df.head(2)

,case_number,year,type,country,species_,date,mes,fatal,sexo,especie_cat,age_limpio
0,2018.06.25,2018,Boating,usa,White shark,25-Jun-2018,Jun,n,F,White,57.0
1,2018.06.03.a,2018,Unprovoked,brazil,Tiger shark,03-Jun-2018,Jun,y,M,Tiger,18.0


In [30]:
df.columns

Index(['case_number', 'year', 'type', 'country', 'species_', 'date', 'mes',
       'fatal', 'sexo', 'especie_cat', 'age_limpio'],
      dtype='object')

In [31]:
df_cat=df.select_dtypes(include="object")
df_cat.stb.missing()

,missing,total,percent
species_,126,1672,7.535885
especie_cat,126,1672,7.535885
fatal,97,1672,5.801435
mes,50,1672,2.990431
sexo,14,1672,0.837321
country,10,1672,0.598086
case_number,0,1672,0.000000
type,0,1672,0.000000
date,0,1672,0.000000


In [33]:
imputer = IterativeImputer(n_nearest_features=None, imputation_order='ascending')

### 1. Es el momento de eliminar los nulos:

1.1 Reemplazad los valores nulos de la columna age por la media de la edad usando el método SimpleImputer.

In [34]:
imputer = SimpleImputer(strategy="mean", missing_values= np.nan)

In [35]:
imputer = imputer.fit(df[["age_limpio"]])

In [36]:
df["age_limpio"] = imputer.transform(df[["age_limpio"]])

In [37]:
df["age_limpio"].isnull().sum()

0

In [38]:
df.head(2)

,case_number,year,type,country,species_,date,mes,fatal,sexo,especie_cat,age_limpio
0,2018.06.25,2018,Boating,usa,White shark,25-Jun-2018,Jun,n,F,White,57.0
1,2018.06.03.a,2018,Unprovoked,brazil,Tiger shark,03-Jun-2018,Jun,y,M,Tiger,18.0


1.2 Reemplazad los valores nulos de la columna sex por la moda, usando el método SimpleImputer.
💡 Pista 💡 La moda en este tipo de aproximación se indica como most_frequent.

In [39]:
df["sexo"].isnull().sum()

14

Hay 14 nulos

In [40]:
imputer2 = SimpleImputer(strategy="most_frequent", missing_values= np.nan)

In [41]:
imputer2 = imputer2.fit(df[["sexo"]])

In [42]:
df["sexo"] = imputer2.transform(df[["sexo"]])

In [43]:
df["sexo"].isnull().sum()

0

1.3 Reemplazad los valores nulos de la columna type por el valor más frecuente (la moda) con el método SimpleImputer.

In [44]:
# No existen valores nulos
df["type"].isnull().sum()

0

In [45]:
# Esta sería la aplicación si existieran nulos
imputer3 = SimpleImputer(strategy="most_frequent", missing_values= np.nan)
imputer3 = imputer3.fit(df[["type"]])
df["type"] = imputer2.transform(df[["type"]])
df["type"].isnull().sum()

0

1.4 Utilizad el método KNN Imputer para reemplazar todos los valores nulos de las columnas numéricas.

In [46]:
df_num = df.select_dtypes(include=np.number)

In [47]:
df_num.columns

Index(['year', 'age_limpio'], dtype='object')

In [48]:
imputerKNN = KNNImputer(n_neighbors=6)
imputerKNN.fit(df_num)

KNNImputer(n_neighbors=6)

In [49]:
cat_numKNN = imputerKNN.transform(df_num)

In [50]:
df_KNN = pd.DataFrame(cat_numKNN, columns=df_num.columns)
df_KNN.head(2)

,year,age_limpio
0,2018.0,57.0
1,2018.0,18.0


In [51]:
columnas_knn = df_KNN.columns 

In [52]:
df.drop(columnas_knn, axis=1, inplace = True)

In [53]:
df[columnas_knn] = df_KNN

In [54]:
df.isnull().sum()

case_number      0
type             0
country         10
species_       126
date             0
mes             50
fatal           97
sexo             0
especie_cat    126
year             0
age_limpio       0
dtype: int64

1.5 Utilizad el método Iterative Imputer para reemplazar todos los valores nulos de las columnas numéricas.

In [55]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [56]:
df2 = pd.read_csv("datos/3.ataques_limpio.csv",index_col=0)

In [57]:
df2.head(3)

,case_number,year,type,country,species_,date,mes,fatal,sexo,especie_cat,age_limpio
0,2018.06.25,2018,Boating,usa,White shark,25-Jun-2018,Jun,n,F,White,57.0
1,2018.06.03.a,2018,Unprovoked,brazil,Tiger shark,03-Jun-2018,Jun,y,M,Tiger,18.0
2,2018.05.26.b,2018,Unprovoked,usa,"Bull shark, 6'",26-May-2018,May,n,M,Bull,15.0


In [58]:
df2.stb.missing()

,missing,total,percent
age_limpio,158,1672,9.449761
species_,126,1672,7.535885
especie_cat,126,1672,7.535885
fatal,97,1672,5.801435
mes,50,1672,2.990431
sexo,14,1672,0.837321
country,10,1672,0.598086
case_number,0,1672,0.000000
year,0,1672,0.000000
type,0,1672,0.000000


In [59]:
numericas = df2.select_dtypes(include = np.number)

In [60]:
numericas.isnull().sum()

year            0
age_limpio    158
dtype: int64

In [61]:
imputer5 = IterativeImputer(n_nearest_features=None, imputation_order='ascending')

In [62]:
imputer5.fit(numericas)

IterativeImputer()

In [63]:
imputer5.transform(numericas)

array([[2018.        ,   57.        ],
       [2018.        ,   18.        ],
       [2018.        ,   15.        ],
       ...,
       [1997.        ,   28.43224178],
       [1997.        ,   28.43224178],
       [1997.        ,   15.        ]])

In [64]:
numericas_it = pd.DataFrame(imputer5.transform(numericas), columns = numericas.columns)

In [65]:
numericas_it.head(2)

,year,age_limpio
0,2018.0,57.0
1,2018.0,18.0


In [66]:
lista_2 = numericas_it.columns

In [67]:
df2.drop(lista_2, axis=1, inplace=True)

In [68]:
df2[lista_2]= numericas_it

In [69]:
numericas_it.stb.missing()

,missing,total,percent
year,0,1672,0.0
age_limpio,0,1672,0.0


In [70]:
df2.stb.missing()

,missing,total,percent
species_,126,1672,7.535885
especie_cat,126,1672,7.535885
fatal,97,1672,5.801435
mes,50,1672,2.990431
sexo,14,1672,0.837321
country,10,1672,0.598086
case_number,0,1672,0.000000
type,0,1672,0.000000
date,0,1672,0.000000
year,0,1672,0.000000


1.6 ¿Podríais explicar qué diferencia hay entre estos dos últimos métodos?

El método KNN imputer reemplaza los valores nulos o perdidos en función de la media de los vecinos no nulos. Le diremos qué número de vecinos deberá tener en cuenta a la hora de calcularla con el parámetro "n_neighbors".

El método Iterative_Imputer en cambio utiliza los datos disponibles en otras columnas con el fin de estimar los valores nulos que se imputarán o que queremos quitar.
Con el parámetro "max_iter" indicamos el número de iteraciones que queremos que realice. Cuantas más iteraciones le indiquemos más probable es que no dejemos información sin considerar. Sin embargo, no hay que poner demasiadas iteraciones porque consume muchos recursos del ordenador y podríamos tener problemas en este sentido. Por defecto el número de rondas que efectúa es de 10 iteraciones.

Existen varios estimadores, pero se considera el Bayesian Ridge como el más adecuado.

### 2. Guardad el csv para seguir trabajando con el en los siguientes ejercicios de pair

En este jupyter hemos creado dos df para trabajar sobre ellos. De los dos, vamos a guardar aquel que tiene menos nulos y por tanto los datos más limpios.

In [71]:
df.head(2)

,case_number,type,country,species_,date,mes,fatal,sexo,especie_cat,year,age_limpio
0,2018.06.25,Boating,usa,White shark,25-Jun-2018,Jun,n,F,White,2018.0,57.0
1,2018.06.03.a,Unprovoked,brazil,Tiger shark,03-Jun-2018,Jun,y,M,Tiger,2018.0,18.0


In [72]:
df.stb.missing()

,missing,total,percent
species_,126,1672,7.535885
especie_cat,126,1672,7.535885
fatal,97,1672,5.801435
mes,50,1672,2.990431
country,10,1672,0.598086
case_number,0,1672,0.000000
type,0,1672,0.000000
date,0,1672,0.000000
sexo,0,1672,0.000000
year,0,1672,0.000000


In [73]:
df2.head(2)

,case_number,type,country,species_,date,mes,fatal,sexo,especie_cat,year,age_limpio
0,2018.06.25,Boating,usa,White shark,25-Jun-2018,Jun,n,F,White,2018.0,57.0
1,2018.06.03.a,Unprovoked,brazil,Tiger shark,03-Jun-2018,Jun,y,M,Tiger,2018.0,18.0


In [74]:
df2.stb.missing()


,missing,total,percent
species_,126,1672,7.535885
especie_cat,126,1672,7.535885
fatal,97,1672,5.801435
mes,50,1672,2.990431
sexo,14,1672,0.837321
country,10,1672,0.598086
case_number,0,1672,0.000000
type,0,1672,0.000000
date,0,1672,0.000000
year,0,1672,0.000000


In [75]:
df.to_csv("datos/5.ataques_limpio.csv")